In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report

%matplotlib inline

sns.set_style('darkgrid')

In [ ]:
data=pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data1=data.drop(['workex','salary','sl_no'],axis=1)

In [ ]:
data1.isna().sum()

In [ ]:
data1

In [ ]:
f,ax=plt.subplots(3,2,figsize=(12,12))

# 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'specialisation'

sns.countplot(data=data1,x='gender',ax=ax[0,0],palette='rainbow',)
sns.countplot(data=data1,x='ssc_b',ax=ax[0,1],palette='rainbow')
sns.countplot(data=data1,x='hsc_b',ax=ax[1,0],palette='rainbow')
sns.countplot(data=data1,x='hsc_s',ax=ax[1,1],palette='rainbow')
sns.countplot(data=data1,x='degree_t',ax=ax[2,0],palette='rainbow')
sns.countplot(data=data1,x='specialisation',ax=ax[2,1],palette='rainbow')


In [ ]:
data1

In [ ]:
data1.columns[data1.dtypes=='object']

In [ ]:
oh = ColumnTransformer(
    [('OHE', OneHotEncoder(sparse=False,drop='first'),
      ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'specialisation'])],
    remainder = 'passthrough'
    )

scaler=StandardScaler()

pipe=Pipeline([('Encoder',oh),('scaler',scaler)])

o1=OneHotEncoder(sparse=False,drop='first')

In [ ]:
x=data1.drop('status',axis=1)
y=data1[['status']]

In [ ]:
x

In [ ]:
y

In [ ]:
y1=o1.fit_transform(y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y1,test_size=0.1,random_state=101)

In [ ]:
x_train=pipe.fit_transform(x_train)
x_test=pipe.transform(x_test)

In [ ]:
print(o1.categories_)
pipe[0].transformers_[0][1].categories_

In [ ]:
lr1=LogisticRegression()
lr1.fit(x_train,y_train.ravel())

pred1=lr1.predict(x_test)

print('classification report')
print(classification_report(y_test,pred1))

In [ ]:
lr2=SVC()
lr2.fit(x_train,y_train.ravel())

pred2=lr2.predict(x_test)

print('classification report')
print(classification_report(y_test,pred2))

In [ ]:
lr3=GridSearchCV(SVC(),param_grid={'gamma':[1,0.1,0.01,0.0001],'C':[1,10,100,1000]},verbose=2,n_jobs=-1)
lr3.fit(x_train,y_train.ravel())

pred3=lr3.predict(x_test)

print('classification report')
print(classification_report(y_test,pred3))

In [ ]:
lr3.best_params_